In [14]:
import tkinter as tk
from tkinter import ttk, messagebox
import hashlib
import datetime
import json
import random
import matplotlib.pyplot as plt
import networkx as nx

# Blockchain Classes
class Block:
    def __init__(self, index, timestamp, data, previous_hash):
        self.index = index
        self.timestamp = timestamp
        self.data = data
        self.previous_hash = previous_hash
        self.hash = self.calculate_hash()

    def calculate_hash(self):
        to_hash = f"{self.index}{self.timestamp}{self.data}{self.previous_hash}"
        return hashlib.sha256(to_hash.encode()).hexdigest()

class Blockchain:
    def __init__(self):
        self.chain = [self.create_genesis_block()]

    def create_genesis_block(self):
        return Block(0, str(datetime.datetime.now()), "Genesis Block", "0")

    def get_latest_block(self):
        return self.chain[-1]

    def add_block(self, data):
        previous_block = self.get_latest_block()
        new_block = Block(len(self.chain), str(datetime.datetime.now()), data, previous_block.hash)
        self.chain.append(new_block)

    def is_chain_valid(self):
        for i in range(1, len(self.chain)):
            current_block = self.chain[i]
            previous_block = self.chain[i - 1]

            if current_block.hash != current_block.calculate_hash():
                return False, i
            if current_block.previous_hash != previous_block.hash:
                return False, i
        return True, -1

    def to_dict(self):
        return [
            {
                "index": block.index,
                "timestamp": block.timestamp,
                "data": block.data,
                "previous_hash": block.previous_hash,
                "hash": block.hash,
            }
            for block in self.chain
        ]

    def from_dict(self, chain_data):
        self.chain = []
        for block_data in chain_data:
            block = Block(
                block_data["index"],
                block_data["timestamp"],
                block_data["data"],
                block_data["previous_hash"],
            )
            block.hash = block_data["hash"]
            self.chain.append(block)

    def get_statistics(self):
        num_blocks = len(self.chain)
        temperatures = [float(block.data.split(",")[0].split(":")[1].strip()) for block in self.chain[1:]]
        avg_temp = sum(temperatures) / len(temperatures) if temperatures else 0
        locations = [block.data.split(",")[1].split(":")[1].strip() for block in self.chain[1:]]
        most_common_location = max(set(locations), key=locations.count) if locations else "N/A"
        return num_blocks, avg_temp, most_common_location

# Tkinter Application
def create_interactive_interface():
    blockchain = Blockchain()

    root = tk.Tk()
    root.title("IoT Blockchain Interface")
    root.geometry("1400x800")
    root.configure(bg="#f0f0f0")

    style = ttk.Style()
    style.configure("TLabel", font=("Arial", 12), background="#f0f0f0")
    style.configure("TButton", font=("Arial", 11), padding=6)
    style.configure("TEntry", padding=5)
    style.configure("Treeview", font=("Arial", 10), rowheight=30)

    

    # Core functionality functions
    def add_data_to_blockchain():
        temperature = temp_entry.get()
        location = location_entry.get()

        if temperature and location:
            try:
                float(temperature)
                data = f"Temperature: {temperature}, Location: {location}"
                blockchain.add_block(data)
                update_blockchain_view()
                temp_entry.delete(0, tk.END)
                location_entry.delete(0, tk.END)
                show_notification("Data added successfully!")
            except ValueError:
                show_notification("Invalid temperature value!", error=True)
        else:
            show_notification("Please fill both fields!", error=True)

    def auto_simulate_data():
        temperature = round(random.uniform(-10, 40), 2)
        location = f"Location-{random.randint(1, 100)}"
        data = f"Temperature: {temperature}, Location: {location}"
        blockchain.add_block(data)
        update_blockchain_view()
        show_notification(f"Simulated data added: {temperature}°C at {location}")

    def update_blockchain_view():
        tree.delete(*tree.get_children())
        for block in blockchain.chain:
            tree.insert("", "end", values=(
                block.index, 
                block.timestamp, 
                block.data, 
                block.hash[:10] + "..."
            ))

    # Validation and file operations
    def validate_blockchain():
        is_valid, invalid_index = blockchain.is_chain_valid()
        if is_valid:
            show_notification("Blockchain validation successful!")
        else:
            show_notification(f"Invalid block detected at index {invalid_index}!", error=True)

    def save_blockchain():
        with open("blockchain_data.json", "w") as file:
            json.dump(blockchain.to_dict(), file, indent=4)
        show_notification("Blockchain saved to file!")

    def load_blockchain():
        try:
            with open("blockchain_data.json", "r") as file:
                chain_data = json.load(file)
                blockchain.from_dict(chain_data)
                update_blockchain_view()
                show_notification("Blockchain loaded successfully!")
        except FileNotFoundError:
            show_notification("No save file found!", error=True)

    # Visualization functions
    def plot_temperature_trend():
        if len(blockchain.chain) < 2:
            show_notification("No temperature data to plot!", error=True)
            return

        temperatures = []
        timestamps = []
        for block in blockchain.chain[1:]:
            try:
                temp = float(block.data.split(",")[0].split(":")[1].strip())
                temperatures.append(temp)
                timestamps.append(block.timestamp)
            except (IndexError, ValueError):
                continue

        plt.figure(figsize=(12, 6))
        plt.plot(timestamps, temperatures, marker='o', linestyle='-', color='#2c7fb8')
        plt.title("Temperature Trend Analysis", fontsize=14)
        plt.xlabel("Timestamp", fontsize=12)
        plt.ylabel("Temperature (°C)", fontsize=12)
        plt.xticks(rotation=45)
        plt.grid(True, alpha=0.3)
        plt.tight_layout()
        plt.show()

    def visualize_blockchain():
        G = nx.DiGraph()
        for block in blockchain.chain:
            G.add_node(block.index, 
                      label=f"Block {block.index}\nHash: {block.hash[:8]}...\nData: {block.data[:15]}...")
            if block.index > 0:
                G.add_edge(block.index-1, block.index)

        plt.figure(figsize=(10, 8))
        pos = nx.spring_layout(G)
        nx.draw(G, pos, with_labels=True, 
               labels=nx.get_node_attributes(G, 'label'),
               node_size=2500, 
               node_color='#66c2a5',
               font_size=8,
               arrowsize=20)
        plt.title("Blockchain Network Visualization")
        plt.show()

    # GUI layout
    input_frame = ttk.LabelFrame(root, text="IoT Data Input", padding=15)
    input_frame.pack(fill="x", padx=10, pady=10)

    ttk.Label(input_frame, text="Temperature (°C):").grid(row=0, column=0, padx=5)
    temp_entry = ttk.Entry(input_frame, width=25)
    temp_entry.grid(row=0, column=1, padx=5)

    ttk.Label(input_frame, text="Location:").grid(row=1, column=0, padx=5)
    location_entry = ttk.Entry(input_frame, width=25)
    location_entry.grid(row=1, column=1, padx=5)

    ttk.Button(input_frame, text="Add Data", command=add_data_to_blockchain).grid(row=2, column=0, pady=10)
    ttk.Button(input_frame, text="Auto Generate", command=auto_simulate_data).grid(row=2, column=1, pady=10)

    # Blockchain display
    tree_frame = ttk.LabelFrame(root, text="Blockchain Records", padding=15)
    tree_frame.pack(fill="both", expand=True, padx=10, pady=10)

    columns = ("Index", "Timestamp", "Data", "Hash")
    tree = ttk.Treeview(tree_frame, columns=columns, show="headings", height=12)
    for col in columns:
        tree.heading(col, text=col)
        tree.column(col, width=100 if col != "Data" else 200, anchor="center")
    tree.pack(fill="both", expand=True)

    # Control panel
    control_frame = ttk.Frame(root, padding=10)
    control_frame.pack(fill="x", padx=10, pady=10)

    buttons = [
        ("Validate Chain", validate_blockchain),
        ("Save Chain", save_blockchain),
        ("Load Chain", load_blockchain),
        ("Temperature Plot", plot_temperature_trend),
        ("Visualize Chain", visualize_blockchain)
    ]

    for idx, (text, command) in enumerate(buttons):
        ttk.Button(control_frame, text=text, command=command).grid(row=0, column=idx, padx=5)

    def show_notification(message, error=False):
        if error:
            messagebox.showerror("Notification", message)
        else:
            messagebox.showinfo("Notification", message)

    update_blockchain_view()
    root.mainloop()

# Run the application
if __name__ == "__main__":
    create_interactive_interface()